<a href="https://colab.research.google.com/github/threegenie/studying_pytorch/blob/main/08_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC%EC%9D%98_%EC%A0%84%EC%B2%98%EB%A6%AC_03_%ED%86%A0%EC%B9%98%ED%85%8D%EC%8A%A4%ED%8A%B8_%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC(kor).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys
import urllib.request

In [2]:
%%capture
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [3]:
import konlpy
from konlpy.tag import Mecab

In [4]:
%%capture
!pip install torchtext

In [5]:
from torchtext.legacy.data import TabularDataset

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7efc6d571b10>)

In [7]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [11]:
# from torchtext import data # torchtext.data 임포트
from torchtext.legacy import data
from konlpy.tag import Mecab

In [9]:
# Mecab을 토크나이저로 사용
tokenizer = Mecab()

In [13]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1



필드(Field)
- 실제 텍스트를 위한 TEXT 객체
- 레이블 데이터를 위한 LABEL 객체
> 필드 인자 정리
```
sequential : 시퀀스 데이터 여부. (True가 기본값)
use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
is_target : 레이블 데이터 여부. (False가 기본값)
fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.
```



In [12]:
# 필드 정의

# id
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

# document
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=20)

# label
LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

#네이버 영화 리뷰 데이터는 3개의 열로 구성되어 있으므로 3개의 필드를 정의